In [1]:
import datetime
import dateutil
import tda
import atexit
import pandas as pd
import requests #Useful when troubleshooting lack of data

In [3]:
#Pull this information from your tda-api setup
API_KEY = '<>@AMER.OAUTHAP'
REDIRECT_URI = 'https://X.X.X.X'
TOKEN_PATH = '<>'

In [4]:
#Initialize webdriver (assumes Chrome and macOS)
def make_webdriver():
    # Import selenium here because it's slow to import
    from selenium import webdriver
    
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options, executable_path=r'/usr/local/Caskroom/chromedriver/83.0.4103.39/chromedriver') #you will need to update the path to chromedriver
    atexit.register(lambda: driver.quit())
    return driver

In [ ]:
# Create a new client
client = tda.auth.easy_client(API_KEY,REDIRECT_URI,TOKEN_PATH,make_webdriver)

In [ ]:
account_id='' #Get the accountID in interest from TOS or TDA website

chromedriver

In [8]:
transaction_type=client.Transactions.TransactionType.TRADE
transactions = client.get_transactions(account_id, transaction_type=transaction_type, symbol=None, start_date=None, end_date=None) #this pulls data without dates prescribed. You can add dates as well to limit your dashboard data
transactions_data=pd.json_normalize(transactions.json())

In [9]:
transactions_data['transactionItem.instrument.optionExpirationDate']=pd.to_datetime(transactions_data['transactionItem.instrument.optionExpirationDate'], infer_datetime_format=True)
transactions_data['settlementDate']=pd.to_datetime(transactions_data.settlementDate, infer_datetime_format=True)
transactions_data['transactionDate']=pd.to_datetime(transactions_data.transactionDate, infer_datetime_format=True)
transactions_data['orderDate']=pd.to_datetime(transactions_data.orderDate, infer_datetime_format=True)
transactions_data['transactionItem.instrument.daysToExpiration']=transactions_data['transactionItem.instrument.optionExpirationDate']-transactions_data['transactionDate']
transactions_data['transactionItem.instrument.daysToExpiration']=transactions_data['transactionItem.instrument.daysToExpiration'].dt.days
transactions_data['fees.totalFee']=transactions_data[['fees.rFee', 'fees.additionalFee', 'fees.cdscFee', 'fees.regFee', 'fees.otherCharges', 'fees.commission', 'fees.optRegFee', 'fees.secFee']].sum(axis=1)

In [10]:
transactions_data.to_pickle('transactions_data_'+account_id)

In [14]:
transactions_data #visualize the data to confirm the data looks right

,type,subAccount,settlementDate,orderId,netAmount,transactionDate,orderDate,transactionSubType,transactionId,cashBalanceEffectFlag,...,transactionItem.positionEffect,transactionItem.instrument.symbol,transactionItem.instrument.underlyingSymbol,transactionItem.instrument.optionExpirationDate,transactionItem.instrument.putCall,transactionItem.instrument.cusip,transactionItem.instrument.description,transactionItem.instrument.assetType,transactionItem.instrument.daysToExpiration,fees.totalFee
0,TRADE,2,2020-08-10,T3163687655,-360.67,2020-08-07 13:32:28,2020-08-07 04:36:18,BY,28085169359,True,...,OPENING,MDLZ_011521C55,MDLZ,2021-01-15 06:00:00,CALL,0MDLZ.AF10055000,MDLZ Jan 15 2021 55.0 Call,OPTION,160.0,0.69
1,TRADE,2,2020-08-07,T3153283181,161.00,2020-08-05 16:06:46,2020-08-05 13:59:26,SL,28029535438,True,...,NaN,HL,NaN,NaT,NaN,422704106,NaN,EQUITY,NaN,0.00
2,TRADE,2,2020-07-27,T3108420608.1,-142.67,2020-07-24 17:51:45,2020-07-24 17:51:45,BY,27797743783,True,...,CLOSING,HL_031921C5,HL,2021-03-19 05:00:00,CALL,0HL...CJ10005000,HL Mar 19 2021 5.0 Call,OPTION,237.0,0.69
3,TRADE,2,2020-07-27,T3108420608,178.33,2020-07-24 17:51:45,2020-07-24 17:51:45,SL,27797743801,True,...,OPENING,HL_012122C5.5,HL,2022-01-21 06:00:00,CALL,0HL...AL20005500,HL Jan 21 2022 5.5 Call,OPTION,545.0,0.69
4,TRADE,2,2020-07-27,T3108367436,177.33,2020-07-24 17:45:56,2020-07-24 17:45:56,SL,27797671802,True,...,OPENING,HL_012122C5.5,HL,2022-01-21 06:00:00,CALL,0HL...AL20005500,HL Jan 21 2022 5.5 Call,OPTION,545.0,0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,TRADE,2,2019-08-15,T2251958638,-390.77,2019-08-14 16:22:23,2019-08-08 01:38:57,BY,23058713097,True,...,OPENING,CCL_011720P45,CCL,2020-01-17 06:00:00,PUT,0CCL..MH00045000,CCL Jan 17 2020 45.0 Put,OPTION,155.0,0.79
99,TRADE,2,2019-08-14,T2250304159,-230.77,2019-08-13 14:19:40,2019-08-07 02:53:10,BY,23049506603,True,...,OPENING,NFG_011720P45,NFG,2020-01-17 06:00:00,PUT,0NFG..MH00045000,NFG Jan 17 2020 45.0 Put,OPTION,156.0,0.79
100,TRADE,2,2019-08-14,T2256815417,-520.77,2019-08-13 14:01:24,2019-08-13 01:29:34,BY,23049359143,True,...,OPENING,PZZA_011720P45,PZZA,2020-01-17 06:00:00,PUT,0PZZA.MH00045000,PZZA Jan 17 2020 45.0 Put,OPTION,156.0,0.79
101,TRADE,2,2019-08-14,T2256813552,-280.77,2019-08-13 13:40:37,2019-08-13 01:25:23,BY,23049052052,True,...,OPENING,PPC_122019C29,PPC,2019-12-20 06:00:00,CALL,0PPC..LK90029000,PPC Dec 20 2019 29.0 Call,OPTION,128.0,0.79
